In [1]:
from main import *

In [2]:
gw_no = 10
league_name = 'rpk'
league_id = get_league_id(league_name)

# functions

In [3]:
def unique_rank_pnl(ranks:list):
    pnl = []
    for i in ranks:
        if i == 1:
            pnl.append(300)
        elif i == 2:
            pnl.append(200)
        elif i == 3:
            pnl.append(100)
        elif i == 4:
            pnl.append(-100)
        elif i == 5:
            pnl.append(-200)
        elif i == 6:
            pnl.append(-300)
    return pnl

def get_winners_ratios_equal_rank(winners_ranks):
    winners_ranks = list(sorted(winners_ranks))
    if winners_ranks == [1,2,3]:
        return {1:3/6, 2:2/6, 3:1/6}
    elif winners_ranks == [1,2,2]:
        return {1:6/12, 2:3/12}
    elif winners_ranks == [1,1,2]:
        return {1:5/12, 2:2/12}
    elif winners_ranks == [1,1,1]:
        return {1:4/12}
    elif winners_ranks == [1,2]:
        return {1:8/12, 2:4/12}
    elif winners_ranks == [1,1]:
        return {1:1/2}
    elif winners_ranks == [1]:
        return {1:1}

def equal_rank_pnl(ranks:list):
    max_rank = max(ranks)
    assert max_rank > 3, 'Max rank < 3 not applicable for this logic'
    pnl = [0]*len(ranks)

    # losers
    for i in range(len(ranks)):
        if ranks[i] == max_rank:
            pnl[i] = -300
        elif ranks[i] == max_rank - 1:
            pnl[i] = -200
        elif ranks[i] == max_rank - 2:
            pnl[i] = -100

    # winners 
    total_prize = sum(pnl) * -1
    # get sorted list of winners' rank
    winners_ranks = [ranks[i] for i in range(len(ranks)) if ranks[i] < max_rank - 2]
    winners_ratio = get_winners_ratios_equal_rank(winners_ranks)
    for i in range(len(pnl)):
        if pnl[i] == 0:
            pnl[i] = round(winners_ratio[ranks[i]] * total_prize)
    assert sum(pnl) == 0
    return pnl

In [4]:
def pnl_rpk(gw):
    gw = gw.copy()
    if max(gw['rank']) == len(gw):
        gw['is_unique_rank'] = True
        gw['pnl'] = unique_rank_pnl(list(gw['rank']))
    else:
        gw['is_unique_rank'] = False
        try:
            gw['pnl'] = equal_rank_pnl(list(gw['rank']))   
        except:
            gw['pnl'] = np.nan
    return gw[gw.pnl != 0].sort_values(by='rank', ascending=True)

# single gw

In [5]:
gw = get_gw_points(league_id, gw_no)

In [6]:
gw

,manager_id,team_name,name,gw_no,points,transfers_cost,h2h_points,rank
0,3498209,TachaponR,Tachapon Ratsameedara,10,79,0,79,1
1,901116,sirawatd,sirawat dulyavit,10,76,0,76,2
2,5386220,pthomas.y,PHOOM T. YENBAMROONG,10,72,0,72,3
5,6798471,proove.club,Natthawat Charoenkitmongkol,10,68,0,68,4
4,1184486,OnkaewmaneeNithiz,Nithiz Onkaewmanee,10,70,4,66,5
3,198741,-ngteam-,Tawiwut Charuwat,10,64,0,64,6


In [7]:
pnl_rpk(gw)

,manager_id,team_name,name,gw_no,points,transfers_cost,h2h_points,rank,is_unique_rank,pnl
0,3498209,TachaponR,Tachapon Ratsameedara,10,79,0,79,1,True,300
1,901116,sirawatd,sirawat dulyavit,10,76,0,76,2,True,200
2,5386220,pthomas.y,PHOOM T. YENBAMROONG,10,72,0,72,3,True,100
5,6798471,proove.club,Natthawat Charoenkitmongkol,10,68,0,68,4,True,-100
4,1184486,OnkaewmaneeNithiz,Nithiz Onkaewmanee,10,70,4,66,5,True,-200
3,198741,-ngteam-,Tawiwut Charuwat,10,64,0,64,6,True,-300


# run multiple gws and combine

In [8]:
df = pd.DataFrame() # if already have csv then pull and append current gw
for gw_no in range(1,1+gw_no):
    df = pd.concat([df, pnl_rpk(get_gw_points(league_id, gw_no))], ignore_index=True)

In [11]:
df.loc[df.gw_no == gw_no]

,manager_id,team_name,name,gw_no,points,transfers_cost,h2h_points,rank,is_unique_rank,pnl
54,3498209,TachaponR,Tachapon Ratsameedara,10,79,0,79,1,True,300
55,901116,sirawatd,sirawat dulyavit,10,76,0,76,2,True,200
56,5386220,pthomas.y,PHOOM T. YENBAMROONG,10,72,0,72,3,True,100
57,6798471,proove.club,Natthawat Charoenkitmongkol,10,68,0,68,4,True,-100
58,1184486,OnkaewmaneeNithiz,Nithiz Onkaewmanee,10,70,4,66,5,True,-200
59,198741,-ngteam-,Tawiwut Charuwat,10,64,0,64,6,True,-300


In [10]:
df.groupby('name')['pnl'].sum().sort_values(ascending=False)

name
Tachapon Ratsameedara          1133
sirawat dulyavit                867
PHOOM T. YENBAMROONG            217
Tawiwut Charuwat                183
Natthawat Charoenkitmongkol   -1100
Nithiz Onkaewmanee            -1300
Name: pnl, dtype: int64